In [3]:
import re
import time
import random
from urllib.parse import urljoin

import requests
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns

In [4]:
df = pd.read_csv(r"..\data\clean\books_merged_clean.csv")

In [5]:
df

,title,author,published_year,language,subjects,cover
0,21st century houses: riba award-winning homes,dominic bradbury,2022,english,"domestic architecture, architecture, awards, m...",https://openlibrary.org/images/icons/avatar_bo...
1,architecture china 2020 building with nature j...,l. xiangning,2022,english,"architecture, awards, sustainable architecture...",https://openlibrary.org/images/icons/avatar_bo...
2,cyberarts 2021: international compendium prix ...,markus jandl,2022,english,"computer art, awards, computer animation, prix...",https://covers.openlibrary.org/b/id/13794706-M...
3,"deutsche bank ""artists of the year"" 2021: maxw...",maxwell alexandre,2022,english,"modern art, exhibitions, art, awards, maxwell ...",https://openlibrary.org/images/icons/avatar_bo...
4,in the shadow of trees,belgium) photobrussels festival (6th 202...,2022,french,"artistic photography, exhibitions, awards, pho...",https://openlibrary.org/images/icons/avatar_bo...
...,...,...,...,...,...,...
1033,premiums offered by the society instituted at ...,"society for the encouragement of arts, manufac...",1772,english,"plants, commerce, art, fisheries, awards, inve...",https://covers.openlibrary.org/b/id/10444430-L...
1034,i pregj delle belle arti,"onofrio alfani, accademia di san luca",1762,italian,"accademia di san luca., art -- awards -- italy...",https://covers.openlibrary.org/b/id/6517671-L.jpg
1035,per la solenne distribuzione del premio alle t...,celestino petracchi,1728,italian,"awards, art, early works to 1800, accademia cl...",https://covers.openlibrary.org/b/id/11908831-L...
1036,il teatro d'onore,ducale collegio de' nobili di parma.,1694,italian,ducale collegio de' nobili di parma -- awards....,https://covers.openlibrary.org/b/id/6074353-L.jpg


In [6]:
# --- Feature Engineering: Weighted Features ---

# Fill NaN values in the text fields
df['title'] = df['title'].fillna('')
df['author'] = df['author'].fillna('')
df['subjects'] = df['subjects'].fillna('')
df['language'] = df['language'].fillna('')

# Create a weighted 'soup' of features
df['soup'] = df['author'].apply(lambda x: ' '.join([x,x,x])) + ' ' + \
             df['title'].apply(lambda x: ' '.join([x,x])) + ' ' + \
             df['language'].apply(lambda x: ' '.join([x,x])) + ' ' + \
             df['subjects']


In [7]:
# --- TF-IDF and Cosine Similarity ---

# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the 'soup' column
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Shape of Cosine Similarity Matrix:", cosine_sim.shape)


Shape of Cosine Similarity Matrix: (1038, 1038)


In [8]:
# --- Book Search Function ---

def find_books_by_keyword(keyword):
    """
    Searches for books containing the keyword and returns a detailed table of the top 5 matches.
    """
    # Search for the keyword (case-insensitive) in the 'soup'
    results = df[df['soup'].str.contains(keyword, case=False, na=False)]

    if results.empty:
        return f"No books found with the keyword: '{keyword}'"

    # Return the first 5 results with detailed columns
    return results[['title', 'author', 'published_year', 'language', 'subjects']].head(5)


In [9]:
# --- Recommendation Function ---

# Create a reverse map of indices and book titles for easy lookup
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    """
    This function takes a book title as input and returns a DataFrame with
    the top 10 most similar books and their details.
    """
    # Get the index of the book that matches the title
    if title not in indices:
        possible_titles = [t for t in indices.index if title in t]
        if not possible_titles:
            return f"Book with title '{title}' not found."
        title = possible_titles[0]
        print(f"Found a partial match, showing recommendations for: '{title}'")

    idx = indices[title]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books (excluding the book itself)
    sim_scores = sim_scores[1:11]

    # Get the book indices and similarity scores
    book_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]

    # Get the top 10 most similar books
    recommendations_df = df.iloc[book_indices][['title', 'author', 'published_year', 'language', 'subjects']]

    # Add the similarity scores to the DataFrame
    recommendations_df['similarity'] = similarity_scores

    return recommendations_df


In [16]:
# --- Example Usage ---

# 1. Search for a book by keyword
print("--- Step 1: Search for a book ---")
search_results = find_books_by_keyword("baseball") # You can change "History" to any keyword
display(search_results)


--- Step 1: Search for a book ---


,title,author,published_year,language,subjects
310,trophy night,ryan howard,2017,english,"juvenile fiction, awards, baseball stories, ch..."
561,"the espn baseball encyclopedia, fifth edition ...","gary gillette, peter gammons",2008,english,"baseball, sports & recreation, sports, basebal..."
823,baseball's best,bob italia,1993,english,baseball players -- biography -- juvenile lite...


In [17]:
# 2. Get recommendations for a book from the search results
print("\n--- Step 2: Get recommendations for a book from the search ---")
if isinstance(search_results, pd.DataFrame) and not search_results.empty:
    # Pick the first book from the search results to get recommendations
    example_title = search_results['title'].iloc[1]
    print(f"\nRecommendations for: '{example_title}'\n")
    recommendations = get_recommendations(example_title)
    display(recommendations) # Use display() for nice table formatting in notebooks
else:
    print("No search results to get recommendations from.")



--- Step 2: Get recommendations for a book from the search ---

Recommendations for: 'the espn baseball encyclopedia, fifth edition (espn baseball encyclopedia)'



,title,author,published_year,language,subjects,similarity
823,baseball's best,bob italia,1993,english,baseball players -- biography -- juvenile lite...,0.476345
310,trophy night,ryan howard,2017,english,"juvenile fiction, awards, baseball stories, ch...",0.202921
696,the calumet collection,judy marchman,2002,english,"sports, race horses, horse racing, sports & re...",0.145379
811,"hockey's heroes 1994 (year in sports, 1994)",bob italia,1994,english,"sports & recreation, juvenile nonfiction, chil...",0.143539
559,tarnished heisman,don yaeger,2008,english,"football - college, sports & recreation / foot...",0.126870
614,the kennedy center american college theater fe...,gary garrison,2006,english,"american drama -- 20th century, american drama...",0.108640
387,the international highrise award 2016,"peter körner, peter cachola schmal",2016,german,"buildings, internationaler hochhaus preis, awa...",0.095878
517,deja blue,new york post,2012,english,"football, awards, football teams, juvenile non...",0.093203
113,best highrises 2018/19: the international high...,"peter cachola schmal, peter korner, maximilian...",2019,"english, german","internationaler hochhaus preis, exhibitions, s...",0.082838
618,what every man wants,andrew mann,2006,english,"sports, awards, sports tournaments",0.078412
